- **Getting User Details**

In [ ]:
import pandas as pd
from github import Github

# GitHub token
g = Github("key")

def get_user_repo_details(username):
    user = g.get_user(username)

    # Get the list of repositories for the user
    repos = user.get_repos()

    # Initialize counters
    total_repos = 0
    created_repos = 0
    forked_repos = 0
    commits_on_created_repos = 0
    commits_on_forked_repos = 0
    forked_users_count = 0  # Counter for forked users

    # Initialize set to store unique forked users
    forked_users = set()

    # Iterate through each repository
    for repo in repos:
        total_repos += 1
        try:
            if repo.fork:
                forked_repos += 1
                # Count the number of commits on forked repos made by the user
                for commit in repo.get_commits():
                    if commit.author and commit.author.login == username:
                        commits_on_forked_repos += 1
                # Add users who forked from this repo to the set
                for fork in repo.get_forks():
                    forked_users.add(fork.owner.login)
            else:
                created_repos += 1
                # Count the number of commits on created repos made by the user
                for commit in repo.get_commits():
                    if commit.author and commit.author.login == username:
                        commits_on_created_repos += 1
        except Exception as e:
            print(f"Error processing repository {repo.full_name}: {e}")

    # Count the number of unique forked users
    forked_users_count = len(forked_users)

    user_repo_details = {
        'login': user.login,
        'name': user.name,
        'bio': user.bio,
        'public_repos': user.public_repos,
        'followers_count': user.followers,
        'following_count': user.following,
        'created_at': user.created_at,
        'updated_at': user.updated_at,
        'avatar_url': user.avatar_url,
        'profile_url': user.html_url,
        'total_repos': total_repos,
        'created_repos': created_repos,
        'forked_repos': forked_repos,
        'commits_on_created_repos': commits_on_created_repos,
        'commits_on_forked_repos': commits_on_forked_repos,
        'forked_users_count': forked_users_count 
    }

    return user_repo_details

# List of usernames
usernames = ["anra8571", "youssefHosni", "SethuSenthil"]

# Initialize a list to store user details
all_user_details = []

# Loop through each username to collect details
for username in usernames:
    user_details = get_user_repo_details(username)
    all_user_details.append(user_details)

# Convert the list of dictionaries to a DataFrame
data = pd.DataFrame(all_user_details)

# Save the DataFrame to a CSV file
data.to_csv('user_details.csv', index=False)

- **Getting the Information from the Commited & Forked Repo**

def flatten_commit_details(commit_list):
    flat_list = []
    for commit_dict in commit_list:
        flat_commit = {}
        for key, value in commit_dict.items():
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    flat_commit[sub_key] = sub_value
            else:
                flat_commit[key] = value
        flat_list.append(flat_commit)
    return flat_list

def get_user_repo_details(username):
    user = g.get_user(username)

    # Get the list of repositories for the user
    repos = user.get_repos()

    # Initialize lists to store commit details
    created_repo_commits = []
    forked_repo_commits = []

    # Iterate through each repository
    for repo in repos:
        try:
            if repo.fork:
                # Count the number of commits on forked repos made by the user
                for commit in repo.get_commits():
                    if commit.author and commit.author.login == username:
                        forked_repo_commits.append({
                            'repo_name': repo.name,
                            'commit': {
                                'commit_date': commit.commit.author.date,
                                'lines_added': commit.stats.additions,
                                'lines_removed': commit.stats.deletions,
                                'change_type': 'Add' if commit.stats.additions > 0 else 'Remove',
                                'language': repo.language
                            }
                        })
            else:
                # Count the number of commits on created repos made by the user
                for commit in repo.get_commits():
                    if commit.author and commit.author.login == username:
                        created_repo_commits.append({
                            'repo_name': repo.name,
                            'commit': {
                                'commit_date': commit.commit.author.date,
                                'lines_added': commit.stats.additions,
                                'lines_removed': commit.stats.deletions,
                                'change_type': 'Add' if commit.stats.additions > 0 else 'Remove',
                                'language': repo.language
                            }
                        })
        except Exception as e:
            print(f"Error processing repository {repo.full_name}: {e}")

    user_repo_details = {
        'created_repo_commits': created_repo_commits,
        'forked_repo_commits': forked_repo_commits
    }

    return user_repo_details

# Example usage
username = "SethuSenthil"
user_details = get_user_repo_details(username)

# Flatten the commit details dictionaries
created_repo_commits_flat = flatten_commit_details(user_details['created_repo_commits'])
forked_repo_commits_flat = flatten_commit_details(user_details['forked_repo_commits'])

# Convert to DataFrame
created_repo_df = pd.DataFrame(created_repo_commits_flat)
forked_repo_df = pd.DataFrame(forked_repo_commits_flat)

print("Created Repo Commits:")
print(created_repo_df)
print("\nForked Repo Commits:")
print(forked_repo_df)

created_repo_df['login']="SethuSenthil"
forked_repo_df['login']="SethuSenthil"

created_repo_df.to_csv("youssefHosni_created_repo.csv", index=False)
forked_repo_df.to_csv('youssefHosni_forked_repo.csv', index=False)

- **Stacking the CSV Files Together.**

In [13]:
import os
import pandas as pd

def stack_csv_files(folder_path):
    # Initialize an empty DataFrame to store the stacked data
    stacked_data = pd.DataFrame()

    # Iterate over each file in the folder
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            # Read the CSV file
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            
            # Concatenate the data to the stacked DataFrame
            stacked_data = pd.concat([stacked_data, df], ignore_index=True)
    
    return stacked_data

# Provide the path to folder containing CSV files
folder_path = 'data\dashboard'
stacked_data = stack_csv_files(folder_path)


print(stacked_data)


<>:21: SyntaxWarning: invalid escape sequence '\d'
<>:21: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Aadhavan\AppData\Local\Temp\ipykernel_586464\1285222807.py:21: SyntaxWarning: invalid escape sequence '\d'
  folder_path = 'data\dashboard'


                repo_name                commit_date  lines_added  \
0     annalrahn.github.io  2023-05-17 06:43:55+00:00          1.0   
1     annalrahn.github.io  2023-05-17 06:39:57+00:00          0.0   
2     annalrahn.github.io  2023-05-17 06:39:25+00:00          1.0   
3     annalrahn.github.io  2023-05-17 06:37:08+00:00          1.0   
4     annalrahn.github.io  2023-05-17 06:36:37+00:00          1.0   
...                   ...                        ...          ...   
3266             localGPT  2023-10-24 18:11:00+00:00          0.0   
3267             localGPT  2023-10-24 18:10:38+00:00          0.0   
3268             localGPT  2023-10-24 18:10:01+00:00         29.0   
3269             localGPT  2023-10-24 18:09:33+00:00         91.0   
3270             localGPT  2023-10-24 18:09:09+00:00          2.0   

      lines_removed change_type language         login repo_nature  
0               1.0         Add     HTML      anra8571     Created  
1               1.0      Remove  

In [12]:
stacked_data.to_csv("commit&forked_data.csv",index=False)